# Build Dataset — Weather Risk Inputs

Notebook para consolidar dados necessários ao projeto em um único dataset diário (POWER, OISST, NHC e IBTrACS).

- Define parâmetros (lat, lon, início, fim)
- Monta dataset com `assemble_dataset`
- Pré-visualiza e salva em CSV/Parquet



In [1]:
# Instalação leve (se necessário)
import sys, subprocess

def pip_install(pkgs):
    try:
        import importlib
        for p in pkgs:
            try:
                importlib.import_module(p.split("[")[0].replace("-","_"))
            except Exception:
                raise
        return
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", *pkgs])

pip_install([
    "pandas>=2.2.0",
    "numpy>=1.26",
    "requests>=2.31",
    "pyarrow>=16.0.0",
    "geopandas>=0.14.0",
    "descartes>=1.1.0",
    "geobr",
])

import pandas as pd
import numpy as np
import geopandas as gpd
import geobr
from datetime import date, timedelta

# Garantir que o módulo atualizado seja recarregado antes dos imports simbólicos
# import importlib, data_fetch as _df # Removido, pois as funções foram movidas
# importlib.reload(_df) # Removido

from load_dataset.ibtracs_data import fetch_ibtracs_all
from load_dataset.open_meteo_data import fetch_open_meteo_daily_forecast, fetch_open_meteo_marine_sst
from load_dataset.power_data import fetch_power_daily



In [2]:
# Parâmetros de data
end = date.today()
start = end - timedelta(days=365 * 8)  # 3 anos

# Lista de pontos para coleta (Latitude, Longitude)
points_of_interest = [
    # Capitais do Brasil
    {'name': 'Aracaju', 'latitude': -10.9111, 'longitude': -37.0717},
    {'name': 'Belém', 'latitude': -1.4558, 'longitude': -48.4900},
    {'name': 'Belo Horizonte', 'latitude': -19.9167, 'longitude': -43.9345},
    {'name': 'Boa Vista', 'latitude': 2.8197, 'longitude': -60.6710},
    {'name': 'Brasília', 'latitude': -15.7797, 'longitude': -47.9297},
    {'name': 'Campo Grande', 'latitude': -20.4428, 'longitude': -54.6464},
    {'name': 'Cuiabá', 'latitude': -15.5961, 'longitude': -56.0967},
    {'name': 'Curitiba', 'latitude': -25.4284, 'longitude': -49.2733},
    {'name': 'Florianópolis', 'latitude': -27.5935, 'longitude': -48.5585},
    {'name': 'Fortaleza', 'latitude': -3.7319, 'longitude': -38.5267},
    {'name': 'Goiânia', 'latitude': -16.6869, 'longitude': -49.2643},
    {'name': 'João Pessoa', 'latitude': -7.1197, 'longitude': -34.8645},
    {'name': 'Macapá', 'latitude': 0.0386, 'longitude': -51.0669},
    {'name': 'Maceió', 'latitude': -9.6659, 'longitude': -35.7359},
    {'name': 'Manaus', 'latitude': -3.1190, 'longitude': -60.0217},
    {'name': 'Natal', 'latitude': -5.7945, 'longitude': -35.2105},
    {'name': 'Palmas', 'latitude': -10.1832, 'longitude': -48.3328},
    {'name': 'Porto Alegre', 'latitude': -30.0346, 'longitude': -51.2177},
    {'name': 'Porto Velho', 'latitude': -8.7619, 'longitude': -63.8314},
    {'name': 'Recife', 'latitude': -8.0476, 'longitude': -34.8769},
    {'name': 'Rio Branco', 'latitude': -9.9723, 'longitude': -67.8106},
    {'name': 'Rio de Janeiro', 'latitude': -22.9068, 'longitude': -43.1729},
    {'name': 'Salvador', 'latitude': -12.9714, 'longitude': -38.5014},
    {'name': 'São Luís', 'latitude': -2.5298, 'longitude': -44.3039},
    {'name': 'São Paulo', 'latitude': -23.5505, 'longitude': -46.6333},
    {'name': 'Teresina', 'latitude': -5.0919, 'longitude': -42.8033},
    {'name': 'Vitória', 'latitude': -20.3150, 'longitude': -40.3128},
    # Outros pontos no Brasil
    {'name': 'Uberlândia', 'latitude': -18.9186, 'longitude': -48.2772},
    # Pontos na Europa
    {'name': 'Londres', 'latitude': 51.5074, 'longitude': -0.1278},
    {'name': 'Paris', 'latitude': 48.8566, 'longitude': 2.3522},
    {'name': 'Berlim', 'latitude': 52.5200, 'longitude': 13.4050},
    {'name': 'Madri', 'latitude': 40.4168, 'longitude': -3.7038},
    {'name': 'Roma', 'latitude': 41.9028, 'longitude': 12.4964},
    {'name': 'Lisboa', 'latitude': 38.7223, 'longitude': -9.1393},
    {'name': 'Amsterdã', 'latitude': 52.3676, 'longitude': 4.9041},
    {'name': 'Moscou', 'latitude': 55.7558, 'longitude': 37.6173},
    {'name': 'Dublin', 'latitude': 53.3498, 'longitude': -6.2603},
    {'name': 'Atenas', 'latitude': 37.9838, 'longitude': 23.7275},
]

# Converter para DataFrame
points_df = pd.DataFrame(points_of_interest)
print(f"Foram selecionados {len(points_df)} pontos de interesse.")
print(points_df.head())


Foram selecionados 38 pontos de interesse.
             name  latitude  longitude
0         Aracaju  -10.9111   -37.0717
1           Belém   -1.4558   -48.4900
2  Belo Horizonte  -19.9167   -43.9345
3       Boa Vista    2.8197   -60.6710
4        Brasília  -15.7797   -47.9297


In [3]:
all_power_dfs = []
all_wx_fc_dfs = []
all_sst_fc_dfs = []

for index, row in points_df.iterrows():
    current_name, current_lat, current_lon = row['name'], row['latitude'], row['longitude']
    print(f"Coletando dados para {current_name} (Lat: {current_lat:.4f}, Lon: {current_lon:.4f})")

    # Fetch individual sources (apenas coleta)
    # print("POWER...") # Comentado para reduzir verbosidade
    power_df = fetch_power_daily(current_lat, current_lon, start, end)
    power_df['name'] = current_name
    power_df['latitude'] = current_lat
    power_df['longitude'] = current_lon
    all_power_dfs.append(power_df)
    # print(power_df.shape) # Comentado para reduzir verbosidade
    # print(power_df.head(3)) # Comentado para reduzir verbosidade

    # Forecast-only fetch
    fc_days = 6 * 30 # 6 meses para frente
    # print("\nWeather forecast (Open-Meteo)...") # Comentado para reduzir verbosidade
    wx_fc = fetch_open_meteo_daily_forecast(current_lat, current_lon, days=fc_days)
    wx_fc['name'] = current_name
    wx_fc['latitude'] = current_lat
    wx_fc['longitude'] = current_lon
    all_wx_fc_dfs.append(wx_fc)
    # print(wx_fc.shape) # Comentado para reduzir verbosidade
    # print(wx_fc.head(3)) # Comentado para reduzir verbosidade

    # print("\nMarine SST forecast (Ope+n-Meteo Marine)...") # Comentado para reduzir verbosidade
    sst_fc = fetch_open_meteo_marine_sst(current_lat, current_lon, days=fc_days)
    sst_fc['name'] = current_name
    sst_fc['latitude'] = current_lat
    sst_fc['longitude'] = current_lon
    all_sst_fc_dfs.append(sst_fc)
    # print(sst_fc.shape) # Comentado para reduzir verbosidade
    # print(sst_fc.head(3)) # Comentado para reduzir verbosidade

# Concatenar todos os dataframes após o loop
final_power_df = pd.concat(all_power_dfs)
final_wx_fc_df = pd.concat(all_wx_fc_dfs)
final_sst_fc_df = pd.concat(all_sst_fc_dfs)


Coletando dados para Aracaju (Lat: -10.9111, Lon: -37.0717)
Coletando dados para Belém (Lat: -1.4558, Lon: -48.4900)
Coletando dados para Belo Horizonte (Lat: -19.9167, Lon: -43.9345)
Coletando dados para Boa Vista (Lat: 2.8197, Lon: -60.6710)
Coletando dados para Brasília (Lat: -15.7797, Lon: -47.9297)
Coletando dados para Campo Grande (Lat: -20.4428, Lon: -54.6464)
Coletando dados para Cuiabá (Lat: -15.5961, Lon: -56.0967)
Coletando dados para Curitiba (Lat: -25.4284, Lon: -49.2733)
Coletando dados para Florianópolis (Lat: -27.5935, Lon: -48.5585)
Coletando dados para Fortaleza (Lat: -3.7319, Lon: -38.5267)
Coletando dados para Goiânia (Lat: -16.6869, Lon: -49.2643)
Coletando dados para João Pessoa (Lat: -7.1197, Lon: -34.8645)
Coletando dados para Macapá (Lat: 0.0386, Lon: -51.0669)
Coletando dados para Maceió (Lat: -9.6659, Lon: -35.7359)
Coletando dados para Manaus (Lat: -3.1190, Lon: -60.0217)
Coletando dados para Natal (Lat: -5.7945, Lon: -35.2105)
Coletando dados para Palmas (L

In [4]:
# Salvamento dos datasets consolidados (CSV)

try:
    final_power_df.to_csv(f"power_MultiLocation_{start}_{end}.csv")
    print("Salvo:", f"power_MultiLocation_{start}_{end}.csv")
except Exception as e:
    print("Erro ao salvar POWER consolidado:", e)

try:
    if not final_wx_fc_df.empty:
        final_wx_fc_df.to_csv(f"forecast_weather_MultiLocation.csv")
        print("Salvo:", f"forecast_weather_MultiLocation.csv")
except Exception as e:
    print("Erro ao salvar forecast weather consolidado:", e)

try:
    if not final_sst_fc_df.empty:
        final_sst_fc_df.to_csv(f"forecast_sst_MultiLocation.csv")
        print("Salvo:", f"forecast_sst_MultiLocation.csv")
except Exception as e:
    print("Erro ao salvar forecast sst consolidado:", e)


Salvo: power_MultiLocation_2017-10-07_2025-10-05.csv
Salvo: forecast_weather_MultiLocation.csv
